In [1]:
import pandas as pd
import datetime
import numpy
import re
from sklearn.naive_bayes import GaussianNB
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn import decomposition, ensemble
from sklearn.ensemble import RandomForestClassifier

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
# Function for calculating age of a person given his date of birth
from datetime import date
def calculate_age(born):
    today = date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

In [3]:
#Function for calculating number of days between two dates 
def datediff(any_date):
    today = date.today()
    datediff = today-any_date 
    return datediff.days

In [4]:
# Calculating the Tenure in months given the data is in X Years and Y months format
def age_length(TenureAsYearsAndMonths):
    years = int(re.findall(r'(\d+)yrs',TenureAsYearsAndMonths )[0])
    months = int(re.findall(r'(\d+)mon',TenureAsYearsAndMonths )[0])
    total_tenure = (years*12)+months
    return total_tenure

In [5]:

# Function for doing all the data manipulations
def AllDataManipulations(MainDataFrame,Manip_Data = pd.DataFrame,temp1 = pd.DataFrame, temp2 = pd.DataFrame,Manip_Completed = pd.DataFrame):
    Manip_Data = MainDataFrame.copy()
    Manip_Data['Employment.Type'].fillna(Manip_Data['Employment.Type'].mode()[0],inplace = True)
    Manip_Data[['loan_default','branch_id','supplier_id','manufacturer_id','Current_pincode_ID','State_ID','Employee_code_ID']] = Manip_Data[
        ['loan_default','branch_id','supplier_id','manufacturer_id','Current_pincode_ID','State_ID','Employee_code_ID']].apply(lambda x: x.astype('category'))
    Manip_Data['Date.of.Birth'] = pd.to_datetime(Manip_Data['Date.of.Birth'])
    Manip_Data['Age'] = Manip_Data['Date.of.Birth'].apply(lambda x: calculate_age(x))
    Manip_Data['DisbursalDate'] = pd.to_datetime(Manip_Data['DisbursalDate'])
    Manip_Data['HowManyDaysSinceDisburse'] = Manip_Data['DisbursalDate'].dt.date.apply(lambda x: datediff(x))
    Manip_Data['AvgAcctAgeInMonths'] = Manip_Data['AVERAGE.ACCT.AGE'].apply(lambda x: age_length(x))
    Manip_Data['CredHistLenInMonts'] = Manip_Data['CREDIT.HISTORY.LENGTH'].apply(lambda x: age_length(x))
    temp1 = pd.get_dummies(Manip_Data['Employment.Type'],prefix='EmploymentType')
    temp2 = pd.get_dummies(Manip_Data['PERFORM_CNS.SCORE.DESCRIPTION'],prefix='Bureau_score_description')
    Manip_Completed = pd.concat([Manip_Data[['disbursed_amount', 'asset_cost', 'ltv', 'branch_id',
       'supplier_id', 'manufacturer_id', 'Current_pincode_ID','Age',
       'HowManyDaysSinceDisburse','State_ID', 'Employee_code_ID',
       'MobileNo_Avl_Flag', 'Aadhar_flag', 'PAN_flag', 'VoterID_flag',
       'Driving_flag', 'Passport_flag', 'PERFORM_CNS.SCORE',
       'PRI.NO.OF.ACCTS', 'PRI.ACTIVE.ACCTS','PRI.OVERDUE.ACCTS', 'PRI.CURRENT.BALANCE', 'PRI.SANCTIONED.AMOUNT',
       'PRI.DISBURSED.AMOUNT', 'SEC.NO.OF.ACCTS', 'SEC.ACTIVE.ACCTS',
       'SEC.OVERDUE.ACCTS', 'SEC.CURRENT.BALANCE', 'SEC.SANCTIONED.AMOUNT',
       'SEC.DISBURSED.AMOUNT', 'PRIMARY.INSTAL.AMT', 'SEC.INSTAL.AMT',
       'NEW.ACCTS.IN.LAST.SIX.MONTHS', 'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS',
       'AvgAcctAgeInMonths','CredHistLenInMonts','NO.OF_INQUIRIES',
       'loan_default']],temp1,temp2],axis=1)
    return Manip_Data,Manip_Completed

In [6]:
MasterData = pd.read_csv("train.csv")
Data1 = MasterData.copy()

In [7]:
Train_ManipData_1,Train_ManipCompleted_1 = AllDataManipulations(Data1)

In [8]:
Train_ManipCompleted_1.columns

Index(['disbursed_amount', 'asset_cost', 'ltv', 'branch_id', 'supplier_id',
       'manufacturer_id', 'Current_pincode_ID', 'Age',
       'HowManyDaysSinceDisburse', 'State_ID', 'Employee_code_ID',
       'MobileNo_Avl_Flag', 'Aadhar_flag', 'PAN_flag', 'VoterID_flag',
       'Driving_flag', 'Passport_flag', 'PERFORM_CNS.SCORE', 'PRI.NO.OF.ACCTS',
       'PRI.ACTIVE.ACCTS', 'PRI.OVERDUE.ACCTS', 'PRI.CURRENT.BALANCE',
       'PRI.SANCTIONED.AMOUNT', 'PRI.DISBURSED.AMOUNT', 'SEC.NO.OF.ACCTS',
       'SEC.ACTIVE.ACCTS', 'SEC.OVERDUE.ACCTS', 'SEC.CURRENT.BALANCE',
       'SEC.SANCTIONED.AMOUNT', 'SEC.DISBURSED.AMOUNT', 'PRIMARY.INSTAL.AMT',
       'SEC.INSTAL.AMT', 'NEW.ACCTS.IN.LAST.SIX.MONTHS',
       'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS', 'AvgAcctAgeInMonths',
       'CredHistLenInMonts', 'NO.OF_INQUIRIES', 'loan_default',
       'EmploymentType_Salaried', 'EmploymentType_Self employed',
       'Bureau_score_description_A-Very Low Risk',
       'Bureau_score_description_B-Very Low Risk',

In [9]:
IndeVectors = Train_ManipCompleted_1[['disbursed_amount', 'asset_cost', 'ltv', 'branch_id', 'supplier_id',
       'manufacturer_id', 'Current_pincode_ID', 'Age',
       'HowManyDaysSinceDisburse', 'State_ID', 'Employee_code_ID',
       'MobileNo_Avl_Flag', 'Aadhar_flag', 'PAN_flag', 'VoterID_flag',
       'Driving_flag', 'Passport_flag', 'PERFORM_CNS.SCORE', 'PRI.NO.OF.ACCTS',
       'PRI.ACTIVE.ACCTS', 'PRI.OVERDUE.ACCTS', 'PRI.CURRENT.BALANCE',
       'PRI.SANCTIONED.AMOUNT', 'PRI.DISBURSED.AMOUNT', 'SEC.NO.OF.ACCTS',
       'SEC.ACTIVE.ACCTS', 'SEC.OVERDUE.ACCTS', 'SEC.CURRENT.BALANCE',
       'SEC.SANCTIONED.AMOUNT', 'SEC.DISBURSED.AMOUNT', 'PRIMARY.INSTAL.AMT',
       'SEC.INSTAL.AMT', 'NEW.ACCTS.IN.LAST.SIX.MONTHS',
       'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS', 'AvgAcctAgeInMonths',
       'CredHistLenInMonts', 'NO.OF_INQUIRIES', 
       'EmploymentType_Salaried', 'EmploymentType_Self employed',
       'Bureau_score_description_A-Very Low Risk',
       'Bureau_score_description_B-Very Low Risk',
       'Bureau_score_description_C-Very Low Risk',
       'Bureau_score_description_D-Very Low Risk',
       'Bureau_score_description_E-Low Risk',
       'Bureau_score_description_F-Low Risk',
       'Bureau_score_description_G-Low Risk',
       'Bureau_score_description_H-Medium Risk',
       'Bureau_score_description_I-Medium Risk',
       'Bureau_score_description_J-High Risk',
       'Bureau_score_description_K-High Risk',
       'Bureau_score_description_L-Very High Risk',
       'Bureau_score_description_M-Very High Risk',
       'Bureau_score_description_No Bureau History Available',
       'Bureau_score_description_Not Scored: More than 50 active Accounts found',
       'Bureau_score_description_Not Scored: No Activity seen on the customer (Inactive)',
       'Bureau_score_description_Not Scored: No Updates available in last 36 months',
       'Bureau_score_description_Not Scored: Not Enough Info available on the customer',
       'Bureau_score_description_Not Scored: Only a Guarantor',
       'Bureau_score_description_Not Scored: Sufficient History Not Available']]
Labels = Train_ManipCompleted_1['loan_default']

In [10]:
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(IndeVectors,Labels,random_state = 1234,test_size = 0.10)

# Random forest
rf_model_1 = RandomForestClassifier(n_estimators=300,bootstrap = True,max_features = 'sqrt')
rf_model_1.fit(train_x,list(train_y))
ypred_rf1 = rf_model_1.predict(valid_x)
auc_rf1 = metrics.roc_auc_score(valid_y,ypred_rf1)
print("AUC of Random forest Model_1: ",auc_rf1)
# Naive Bayes
gnb = GaussianNB()
gnb.fit(train_x,train_y)
ypred_gnb = gnb.predict(valid_x)
auc_gnb = metrics.roc_auc_score(valid_y,ypred_gnb)
print("AUC of Naive Bayes Model_1: ",auc_gnb)

In [11]:
# Modifying data manipulations function to work for test data as it doesn't have labels column
# Function for doing all the data manipulations
def AllDataManipulations_forTest(MainDataFrame,Manip_Data = pd.DataFrame,temp1 = pd.DataFrame, temp2 = pd.DataFrame,Manip_Completed = pd.DataFrame):
    Manip_Data = MainDataFrame.copy()
    Manip_Data['Employment.Type'].fillna(Manip_Data['Employment.Type'].mode()[0],inplace = True)
    Manip_Data[['branch_id','supplier_id','manufacturer_id','Current_pincode_ID','State_ID','Employee_code_ID']] = Manip_Data[
        ['branch_id','supplier_id','manufacturer_id','Current_pincode_ID','State_ID','Employee_code_ID']].apply(lambda x: x.astype('category'))
    Manip_Data['Date.of.Birth'] = pd.to_datetime(Manip_Data['Date.of.Birth'])
    Manip_Data['Age'] = Manip_Data['Date.of.Birth'].apply(lambda x: calculate_age(x))
    Manip_Data['DisbursalDate'] = pd.to_datetime(Manip_Data['DisbursalDate'])
    Manip_Data['HowManyDaysSinceDisburse'] = Manip_Data['DisbursalDate'].dt.date.apply(lambda x: datediff(x))
    Manip_Data['AvgAcctAgeInMonths'] = Manip_Data['AVERAGE.ACCT.AGE'].apply(lambda x: age_length(x))
    Manip_Data['CredHistLenInMonts'] = Manip_Data['CREDIT.HISTORY.LENGTH'].apply(lambda x: age_length(x))
    temp1 = pd.get_dummies(Manip_Data['Employment.Type'],prefix='EmploymentType')
    temp2 = pd.get_dummies(Manip_Data['PERFORM_CNS.SCORE.DESCRIPTION'],prefix='Bureau_score_description')
    Manip_Completed = pd.concat([Manip_Data[['disbursed_amount', 'asset_cost', 'ltv', 'branch_id',
       'supplier_id', 'manufacturer_id', 'Current_pincode_ID','Age',
       'HowManyDaysSinceDisburse','State_ID', 'Employee_code_ID',
       'MobileNo_Avl_Flag', 'Aadhar_flag', 'PAN_flag', 'VoterID_flag',
       'Driving_flag', 'Passport_flag', 'PERFORM_CNS.SCORE',
       'PRI.NO.OF.ACCTS', 'PRI.ACTIVE.ACCTS','PRI.OVERDUE.ACCTS', 'PRI.CURRENT.BALANCE', 'PRI.SANCTIONED.AMOUNT',
       'PRI.DISBURSED.AMOUNT', 'SEC.NO.OF.ACCTS', 'SEC.ACTIVE.ACCTS',
       'SEC.OVERDUE.ACCTS', 'SEC.CURRENT.BALANCE', 'SEC.SANCTIONED.AMOUNT',
       'SEC.DISBURSED.AMOUNT', 'PRIMARY.INSTAL.AMT', 'SEC.INSTAL.AMT',
       'NEW.ACCTS.IN.LAST.SIX.MONTHS', 'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS',
       'AvgAcctAgeInMonths','CredHistLenInMonts','NO.OF_INQUIRIES']],temp1,temp2],axis=1)
    return Manip_Data,Manip_Completed


In [12]:
Master_test = pd.read_csv("test_bqCt9Pv.csv")
Test_Data = Master_test.copy()
Test_ManipData,Test_ManipCompleted = AllDataManipulations_forTest(Test_Data)

In [13]:
Test_Data.head()

,UniqueID,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,Date.of.Birth,Employment.Type,...,SEC.CURRENT.BALANCE,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,AVERAGE.ACCT.AGE,CREDIT.HISTORY.LENGTH,NO.OF_INQUIRIES
0,655269,53478,63558,86.54,67,22807,45,1497,01-01-74,Salaried,...,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0
1,723482,55513,63163,89.45,67,22807,45,1497,20-05-85,Self employed,...,0,0,0,5605,0,1,0,0yrs 8mon,1yrs 0mon,1
2,758529,65282,84320,79.93,78,23135,86,2071,14-10-95,Salaried,...,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0
3,763449,46905,63896,76.58,78,17014,45,2070,01-06-73,Self employed,...,0,0,0,0,0,0,0,2yrs 5mon,2yrs 5mon,0
4,708663,51428,63896,86.08,78,17014,45,2069,01-06-72,Salaried,...,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0


In [14]:
Test_ManipCompleted.columns

Index(['disbursed_amount', 'asset_cost', 'ltv', 'branch_id', 'supplier_id',
       'manufacturer_id', 'Current_pincode_ID', 'Age',
       'HowManyDaysSinceDisburse', 'State_ID', 'Employee_code_ID',
       'MobileNo_Avl_Flag', 'Aadhar_flag', 'PAN_flag', 'VoterID_flag',
       'Driving_flag', 'Passport_flag', 'PERFORM_CNS.SCORE', 'PRI.NO.OF.ACCTS',
       'PRI.ACTIVE.ACCTS', 'PRI.OVERDUE.ACCTS', 'PRI.CURRENT.BALANCE',
       'PRI.SANCTIONED.AMOUNT', 'PRI.DISBURSED.AMOUNT', 'SEC.NO.OF.ACCTS',
       'SEC.ACTIVE.ACCTS', 'SEC.OVERDUE.ACCTS', 'SEC.CURRENT.BALANCE',
       'SEC.SANCTIONED.AMOUNT', 'SEC.DISBURSED.AMOUNT', 'PRIMARY.INSTAL.AMT',
       'SEC.INSTAL.AMT', 'NEW.ACCTS.IN.LAST.SIX.MONTHS',
       'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS', 'AvgAcctAgeInMonths',
       'CredHistLenInMonts', 'NO.OF_INQUIRIES', 'EmploymentType_Salaried',
       'EmploymentType_Self employed',
       'Bureau_score_description_A-Very Low Risk',
       'Bureau_score_description_B-Very Low Risk',
       'Bureau_

In [15]:
Test_ManipCompleted['Bureau_score_description_Not Scored: More than 50 active Accounts found'] = 0

In [16]:
Test_ManipCompleted['Bureau_score_description_Not Scored: More than 50 active Accounts found'].unique()

array([0], dtype=int64)

ActualTestPred_rf1 = rf_model_1.predict(Test_ManipCompleted)
ActualTestPred_nb = gnb.predict(Test_ManipCompleted)
RF1_Submissions = pd.concat([Test_Data['UniqueID'],pd.DataFrame(ActualTestPred_rf1,columns=['loan_default'])],axis=1)
nb_Submissions = pd.concat([Test_Data['UniqueID'],pd.DataFrame(ActualTestPred_nb,columns=['loan_default'])],axis=1)

svm_1 = svm.SVC(kernel='linear', C=1)
svm_1.fit(train_x,list(train_y))

ypred_svm1 = svm_1.predict(valid_x)
auc_svm1 = metrics.roc_auc_score(valid_y,ypred_svm1)

In [17]:
########################################################################
# Trials from home
###########################################################################

In [35]:
Supplier_Master = Train_ManipCompleted_1.copy()

In [37]:
def Supplier_DefaulterFinder(TotalDataFrame,tempDf_1=pd.DataFrame(),tempDf_2 =pd.DataFrame(),tempDf_3 = pd.DataFrame()):
    tempDf_1 = pd.DataFrame(TotalDataFrame['supplier_id'].value_counts())
    tempDf_1.reset_index(level=0, inplace=True)
    tempDf_1.columns = ['supplier_id','TotalNoOfVehiclesSold']
    tempDf_2 = pd.DataFrame(TotalDataFrame.groupby('supplier_id')['loan_default'].value_counts())
    tempDf_2.reset_index(level=0, inplace=True)
    tempDf_2.columns = ['supplier_id','NoOfDefaulted_Sales']
    tempDf_2.reset_index(level=0, inplace=True)
    tempDf_2 = tempDf_2[(tempDf_2['loan_default'] == 1)]
    tempDf3 = pd.merge(tempDf_2, tempDf_1, how='right', on= ['supplier_id','supplier_id'],
                       left_index=False, right_index=False)
    #tempDf3 = tempDf3[['supplier_id','TotalNoOfVehiclesSold','NoOfDefaulted_Sales']]
    tempDf3['Defaults%_DueToSupplier'] = ((tempDf3['NoOfDefaulted_Sales'])/(tempDf3['TotalNoOfVehiclesSold']))*100
    return tempDf3

In [38]:
df4 = Supplier_DefaulterFinder(Supplier_Master)

ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long long'

Exception ignored in: 'pandas._libs.lib.is_bool_array'
ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long long'


In [42]:
df4.isnull().sum()

loan_default               287
supplier_id                  0
NoOfDefaulted_Sales        287
TotalNoOfVehiclesSold        0
Defaults%_DueToSupplier    287
dtype: int64

In [43]:
df4['Defaults%_DueToSupplier'].fillna(0,inplace = True)

In [44]:
df4.isnull().sum()

loan_default               287
supplier_id                  0
NoOfDefaulted_Sales        287
TotalNoOfVehiclesSold        0
Defaults%_DueToSupplier      0
dtype: int64

In [45]:
Train_ManipCompleted_2 = pd.merge(Train_ManipCompleted_1,df4,how ='left', left_on='supplier_id',right_on='supplier_id')

ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long long'

Exception ignored in: 'pandas._libs.lib.is_bool_array'
ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long long'


ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long long'

Exception ignored in: 'pandas._libs.lib.is_bool_array'
ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long long'


ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long long'

...

Exception ignored in: 'pandas._libs.lib.is_bool_array'
ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long long'


ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long long'

Exception ignored in: 'pandas._libs.lib.is_bool_array'
ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long long'


ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long long'

Exception ignored in: 'pandas._libs.lib.is_bool_array'
ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long long'


In [46]:
Train_ManipCompleted_2.isnull().sum()

disbursed_amount                                                                       0
asset_cost                                                                             0
ltv                                                                                    0
branch_id                                                                              0
supplier_id                                                                            0
manufacturer_id                                                                        0
Current_pincode_ID                                                                     0
Age                                                                                    0
HowManyDaysSinceDisburse                                                               0
State_ID                                                                               0
Employee_code_ID                                                                       0
MobileNo_Avl_Flag    

In [47]:
len(Train_ManipCompleted_2.columns)

64

In [48]:
Train_ManipCompleted_2.columns

Index(['disbursed_amount', 'asset_cost', 'ltv', 'branch_id', 'supplier_id',
       'manufacturer_id', 'Current_pincode_ID', 'Age',
       'HowManyDaysSinceDisburse', 'State_ID', 'Employee_code_ID',
       'MobileNo_Avl_Flag', 'Aadhar_flag', 'PAN_flag', 'VoterID_flag',
       'Driving_flag', 'Passport_flag', 'PERFORM_CNS.SCORE', 'PRI.NO.OF.ACCTS',
       'PRI.ACTIVE.ACCTS', 'PRI.OVERDUE.ACCTS', 'PRI.CURRENT.BALANCE',
       'PRI.SANCTIONED.AMOUNT', 'PRI.DISBURSED.AMOUNT', 'SEC.NO.OF.ACCTS',
       'SEC.ACTIVE.ACCTS', 'SEC.OVERDUE.ACCTS', 'SEC.CURRENT.BALANCE',
       'SEC.SANCTIONED.AMOUNT', 'SEC.DISBURSED.AMOUNT', 'PRIMARY.INSTAL.AMT',
       'SEC.INSTAL.AMT', 'NEW.ACCTS.IN.LAST.SIX.MONTHS',
       'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS', 'AvgAcctAgeInMonths',
       'CredHistLenInMonts', 'NO.OF_INQUIRIES', 'loan_default_x',
       'EmploymentType_Salaried', 'EmploymentType_Self employed',
       'Bureau_score_description_A-Very Low Risk',
       'Bureau_score_description_B-Very Low Risk

In [49]:
Train_ManipCompleted_2 = Train_ManipCompleted_2[['disbursed_amount', 'asset_cost', 'ltv', 'branch_id', 'supplier_id',
       'manufacturer_id', 'Current_pincode_ID', 'Age',
       'HowManyDaysSinceDisburse', 'State_ID', 'Employee_code_ID',
       'MobileNo_Avl_Flag', 'Aadhar_flag', 'PAN_flag', 'VoterID_flag',
       'Driving_flag', 'Passport_flag', 'PERFORM_CNS.SCORE', 'PRI.NO.OF.ACCTS',
       'PRI.ACTIVE.ACCTS', 'PRI.OVERDUE.ACCTS', 'PRI.CURRENT.BALANCE',
       'PRI.SANCTIONED.AMOUNT', 'PRI.DISBURSED.AMOUNT', 'SEC.NO.OF.ACCTS',
       'SEC.ACTIVE.ACCTS', 'SEC.OVERDUE.ACCTS', 'SEC.CURRENT.BALANCE',
       'SEC.SANCTIONED.AMOUNT', 'SEC.DISBURSED.AMOUNT', 'PRIMARY.INSTAL.AMT',
       'SEC.INSTAL.AMT', 'NEW.ACCTS.IN.LAST.SIX.MONTHS',
       'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS', 'AvgAcctAgeInMonths',
       'CredHistLenInMonts', 'NO.OF_INQUIRIES', 'loan_default_x',
       'EmploymentType_Salaried', 'EmploymentType_Self employed',
       'Bureau_score_description_A-Very Low Risk',
       'Bureau_score_description_B-Very Low Risk',
       'Bureau_score_description_C-Very Low Risk',
       'Bureau_score_description_D-Very Low Risk',
       'Bureau_score_description_E-Low Risk',
       'Bureau_score_description_F-Low Risk',
       'Bureau_score_description_G-Low Risk',
       'Bureau_score_description_H-Medium Risk',
       'Bureau_score_description_I-Medium Risk',
       'Bureau_score_description_J-High Risk',
       'Bureau_score_description_K-High Risk',
       'Bureau_score_description_L-Very High Risk',
       'Bureau_score_description_M-Very High Risk',
       'Bureau_score_description_No Bureau History Available',
       'Bureau_score_description_Not Scored: More than 50 active Accounts found',
       'Bureau_score_description_Not Scored: No Activity seen on the customer (Inactive)',
       'Bureau_score_description_Not Scored: No Updates available in last 36 months',
       'Bureau_score_description_Not Scored: Not Enough Info available on the customer',
       'Bureau_score_description_Not Scored: Only a Guarantor',
       'Bureau_score_description_Not Scored: Sufficient History Not Available','Defaults%_DueToSupplier']]

In [51]:
Indevectors_fe_2 = Train_ManipCompleted_2[['disbursed_amount', 'asset_cost', 'ltv', 'branch_id', 'supplier_id',
       'manufacturer_id', 'Current_pincode_ID', 'Age',
       'HowManyDaysSinceDisburse', 'State_ID', 'Employee_code_ID',
       'MobileNo_Avl_Flag', 'Aadhar_flag', 'PAN_flag', 'VoterID_flag',
       'Driving_flag', 'Passport_flag', 'PERFORM_CNS.SCORE', 'PRI.NO.OF.ACCTS',
       'PRI.ACTIVE.ACCTS', 'PRI.OVERDUE.ACCTS', 'PRI.CURRENT.BALANCE',
       'PRI.SANCTIONED.AMOUNT', 'PRI.DISBURSED.AMOUNT', 'SEC.NO.OF.ACCTS',
       'SEC.ACTIVE.ACCTS', 'SEC.OVERDUE.ACCTS', 'SEC.CURRENT.BALANCE',
       'SEC.SANCTIONED.AMOUNT', 'SEC.DISBURSED.AMOUNT', 'PRIMARY.INSTAL.AMT',
       'SEC.INSTAL.AMT', 'NEW.ACCTS.IN.LAST.SIX.MONTHS',
       'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS', 'AvgAcctAgeInMonths',
       'CredHistLenInMonts', 'NO.OF_INQUIRIES',
       'EmploymentType_Salaried', 'EmploymentType_Self employed',
       'Bureau_score_description_A-Very Low Risk',
       'Bureau_score_description_B-Very Low Risk',
       'Bureau_score_description_C-Very Low Risk',
       'Bureau_score_description_D-Very Low Risk',
       'Bureau_score_description_E-Low Risk',
       'Bureau_score_description_F-Low Risk',
       'Bureau_score_description_G-Low Risk',
       'Bureau_score_description_H-Medium Risk',
       'Bureau_score_description_I-Medium Risk',
       'Bureau_score_description_J-High Risk',
       'Bureau_score_description_K-High Risk',
       'Bureau_score_description_L-Very High Risk',
       'Bureau_score_description_M-Very High Risk',
       'Bureau_score_description_No Bureau History Available',
       'Bureau_score_description_Not Scored: More than 50 active Accounts found',
       'Bureau_score_description_Not Scored: No Activity seen on the customer (Inactive)',
       'Bureau_score_description_Not Scored: No Updates available in last 36 months',
       'Bureau_score_description_Not Scored: Not Enough Info available on the customer',
       'Bureau_score_description_Not Scored: Only a Guarantor',
       'Bureau_score_description_Not Scored: Sufficient History Not Available','Defaults%_DueToSupplier']]
Lables_fe_2 = Train_ManipCompleted_2['loan_default_x']

In [52]:
trainfe2_x,validfe2_x,trainfe2_y,validfe2_y = model_selection.train_test_split(Indevectors_fe_2,Lables_fe_2,random_state = 1234, test_size = 0.10)

In [53]:
print(len(trainfe2_x))
print(len(trainfe2_y))
print(len(validfe2_x))
print(len(validfe2_y))

209838
209838
23316
23316


In [56]:
rf_fe2 = RandomForestClassifier(n_estimators=300,bootstrap = False, max_features=60)
rf_fe2.fit(trainfe2_x,list(trainfe2_y))

RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
            max_depth=None, max_features=60, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [57]:
ypred_rf_fe2 = rf_fe2.predict(validfe2_x)
print("AUC of Random Forest model after second Feature Engg. : ",metrics.roc_auc_score(list(validfe2_y),ypred_rf_fe2))

AUC of Random Forest model after second Feature Engg. :  0.5385898168263569
